<font size="+3"><strong>2.1. Predicting Price with Size</strong></font>

In [2]:

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.utils.validation import check_is_fitted


**Task 2.1.1:** Write a function named `wrangle` that takes a file path as an argument and returns a DataFrame.

In [3]:
def wrangle(filepath):
    # Read CSV file into DataFrame
    df = pd.read_csv(filepath)
    
    # Subset to properties in "Capital Federal"
    mask_ba = df['place_with_parent_names'].str.contains('Capital Federal')
    
    # Subset to "apartments"
    mask_apt = df['property_type'] == 'apartment'
    
    # Subset to properties where 'price_aprox_usd' < 400000
    mask_price = df['price_aprox_usd'] < 400000
    
    # Subset
    df = df[mask_ba & mask_apt & mask_price]
    
    # Remove outliers by 'surface_covered_in_m2'
    low, high = df['surface_covered_in_m2'].quantile([0.1, 0.9])
    mask_area = df['surface_covered_in_m2'].between(low, high)
    
    df = df[mask_area]
    
    return df